# core

> Create a list of symbols in a python package

In [ ]:
#| default_exp core

In [ ]:
#|export
import importlib
import pkgutil
from astroid import MANAGER, FunctionDef, ClassDef
from fastcore.utils import Path
from fastcore.script import call_parse, store_false, store_true, Param

In [ ]:
#|hide
from fastcore.test import test_eq
from astroid import parse, extract_node

## Format Symbols

Once you collect symbols, you want to format it as a markdown list

In [ ]:
#| export
def format_symbol(name, signature, doc, decorators=None, is_method=False):
    "format the information in markdown"
    params = signature.split('(', 1)[1].rsplit(')', 1)[0] if '(' in signature else ''
    decorator_str = ' '.join(f'@{d}' for d in decorators) + ' ' if decorators else ''
    formatted = f"- `{decorator_str.strip()}{' ' if decorator_str else ''}{'def ' if not is_method else ''}{name}({params})`\n"
    if doc:
        doc_lines = doc.strip().split('\n')
        formatted += '    ' + '\n    '.join(doc_lines) + '\n'
    return formatted

In [ ]:
#|hide
test_eq(format_symbol("hello", "hello()", "This is a test function"), "- `def hello()`\n    This is a test function\n")

# Test method formatting
test_eq(format_symbol("world", "world(self)", "A method", is_method=True), "- `world(self)`\n    A method\n")

# Test with decorators
test_eq(format_symbol("decorated", "decorated()", "Decorated function", decorators=["decorator1", "decorator2"]), "- `@decorator1 @decorator2 def decorated()`\n    Decorated function\n")

# Test with parameters
test_eq(format_symbol("params", "params(a, b, c=None)", "Function with parameters"), "- `def params(a, b, c=None)`\n    Function with parameters\n")

# Test with multi-line docstring
test_eq(format_symbol("multiline", "multiline()", "First line\nSecond line\nThird line"), "- `def multiline()`\n    First line\n    Second line\n    Third line\n")

# Test with no docstring
test_eq(format_symbol("no_doc", "no_doc()", ""), "- `def no_doc()`\n")

# Test method with decorators and parameters
test_eq(format_symbol("complex", "complex(self, x, y=0)", "Complex method", decorators=["classmethod"], is_method=True), "- `@classmethod complex(self, x, y=0)`\n    Complex method\n")

## Parse Symbols In The Module

Next we parse symbols in the module that we want to list.

In [ ]:
#| export
def is_public_symbol(name): return not name.startswith('_') or (name.startswith('__') and name.endswith('__'))
def is_valid_method(method, method_name): return isinstance(method, FunctionDef) and is_public_symbol(method_name)
def get_decorators(obj): return [d.as_string() for d in obj.decorators.nodes] if obj.decorators else []
def log_error(name, error): raise RuntimeError(f"Error processing symbol {name}: {str(error)}")

In [ ]:
#|export
def get_params(func):
    params = []
    for arg in func.args.args: params.append(arg.name)
    if func.args.vararg: params.append(f"*{func.args.vararg}")
    if func.args.kwarg: params.append(f"**{func.args.kwarg}")
    return ', '.join(params)

In [ ]:
#|export
def process_function(func, name, include_no_docstring):
    "Parse functions"
    params = get_params(func)
    signature = f"{name}({params})"
    doc = func.doc_node.value if func.doc_node else ""
    decorators = get_decorators(func)
    if include_no_docstring or doc:
        return ('function', name, signature, doc, decorators)
    return None

In [ ]:
#|hide
func_with_doc = parse('''
def test_func(a, b):
    """This is a test function"""
    pass
''').body[0]

result = process_function(func_with_doc, 'test_func', False)
test_eq(result, ('function', 'test_func', 'test_func(a, b)', 'This is a test function', []))

In [ ]:
#|export
def _process_method(method, method_name):
    method_params = get_params(method)
    method_signature = f"{method_name}({method_params})"
    method_doc = method.doc_node.value if method.doc_node else ""
    method_decorators = get_decorators(method)
    return (method_name, method_signature, method_doc, method_decorators)

def process_class(cls, name, include_no_docstring):
    "Parse classes."
    class_doc = cls.doc_node.value if cls.doc_node else ""
    class_decorators = get_decorators(cls)
    methods = [_process_method(method, method_name) 
               for method_name, method in cls.items() 
               if is_valid_method(method, method_name)]
    return ('class', name, class_doc, class_decorators, methods)

For example, this is how a Class will be parsed:

In [ ]:
mock_class = extract_node('''
@decorator
class TestClass:
    """
    Class docstring
    with multiple lines
    """
    @staticmethod
    def method1(arg1):
        """
        Method1 docstring
        with multiple lines
        """
        pass
    def method2(self):
        """Single line docstring"""
        pass
''')

# Process the class
result = process_class(mock_class, "TestClass", True)

# Test equality
test_eq(result, (
    'class',
    'TestClass',
    '\n    Class docstring\n    with multiple lines\n    ',
    ['decorator'],
    [
        ('method1', 'method1(arg1)', '\n        Method1 docstring\n        with multiple lines\n        ', ['staticmethod']),
        ('method2', 'method2(self)', 'Single line docstring', [])
    ]
))

In [ ]:
#|export
def get_public_symbols(module, include_no_docstring):
    "Extract all public symbols"
    symbols = []
    for name, obj in module.items():
        if is_public_symbol(name):
            try:
                if isinstance(obj, FunctionDef):
                    symbol = process_function(obj, name, include_no_docstring)
                    if symbol: symbols.append(symbol)
                elif isinstance(obj, ClassDef):
                    symbols.append(process_class(obj, name, include_no_docstring))
            except Exception as e: log_error(name, e)
    return symbols

In [ ]:
#|hide
test_module = parse('''
def public_function():
    """This is a public function"""
    pass

def _private_function(): pass

class PublicClass:
    """This is a public class"""
    def method(self):
        pass
    def _private_method(): pass

_private_function = lambda: None
''')

# Test 1: Get public symbols without including symbols that lack docstrings
result = get_public_symbols(test_module, False)

# Should have 2 public symbols
test_eq(len(result), 2)
# First item should be public_function
test_eq(result[0][1], 'public_function')
# Second item should be PublicClass
test_eq(result[1][1], 'PublicClass')

# Test 2: Get public symbols including symbols that lack docstrings
result_with_no_docstring = get_public_symbols(test_module, True)

# Should still have 2 public symbols
test_eq(len(result_with_no_docstring), 2)
# PublicClass should have 1 public method
test_eq(len(result_with_no_docstring[1][4]), 1)
# PublicClass should include 'method'
test_eq(result_with_no_docstring[1][4][0][0], 'method')

In [ ]:
#|export
def generate_markdown(package_name, include_no_docstring, verbose=False):
    markdown = [f"# {package_name} Module Documentation\n\n"]
    
    try: package = importlib.import_module(package_name)
    except ImportError: raise ImportError(f"Could not import package {package_name}. Is it installed?")

    for _, module_name, _ in pkgutil.walk_packages(package.__path__, package.__name__ + '.'):
        try:
            if verbose: print(f"Processing module: {module_name}")
            module = MANAGER.ast_from_module_name(module_name)
            symbols = get_public_symbols(module, include_no_docstring)
            
            if symbols:
                markdown.append(f"## {module_name}\n\n")
                module_doc = module.doc_node.value if module.doc_node else ""
                
                if module_doc:
                    markdown.append("> " + "\n> ".join(module_doc.strip().split('\n')) + "\n\n")
                
                for symbol in symbols:
                    if symbol[0] == 'function':
                        _, name, signature, doc, decorators = symbol
                        decorator_str = ' '.join(f'@{d}' for d in decorators)
                        markdown.append(f"- `{decorator_str + ' ' if decorator_str else ''}def {signature}`\n")
                        if doc:
                            markdown.append(f"    {doc.strip()}\n\n")
                    elif symbol[0] == 'class':
                        _, name, class_doc, class_decorators, methods = symbol
                        decorator_str = ' '.join(f'@{d}' for d in class_decorators)
                        markdown.append(f"- `{decorator_str + ' ' if decorator_str else ''}class {name}`\n")
                        if class_doc:
                            markdown.append(f"    {class_doc.strip()}\n\n")
                        for method_name, method_signature, method_doc, method_decorators in methods:
                            method_decorator_str = ' '.join(f'@{d}' for d in method_decorators)
                            markdown.append(f"    - `{method_decorator_str + ' ' if method_decorator_str else ''}def {method_signature}`\n")
                            if method_doc:
                                markdown.append(f"        {method_doc.strip()}\n\n")
                        markdown.append("\n")
            else:
                if verbose: print(f"No public symbols found in {module_name}")
        except Exception as e:
            raise RuntimeError(f"Error processing {module_name}: {str(e)}")
        
    return ''.join(markdown)

Here is a preview of the `fastcore` library, for instance:

In [ ]:
_md = generate_markdown('fastcore', False)
_lns = _md.splitlines()
print('\n'.join([x for x in _lns][:155]))

# fastcore Module Documentation

## fastcore.basics

> Basic functionality used in the fastai library

- `def ifnone(a, b)`
    `b` if `a` is None else `a`

- `def maybe_attr(o, attr)`
    `getattr(o,attr,o)`

- `def basic_repr(flds)`
    Minimal `__repr__`

- `def is_array(x)`
    `True` if `x` supports `__array__` or `iloc`

- `def listify(o, *rest)`
    Convert `o` to a `list`

- `def tuplify(o, use_list, match)`
    Make `o` a tuple

- `def true(x)`
    Test whether `x` is truthy; collections with >0 elements are considered `True`

- `class NullType`
    An object that is `False` and can be called, chained, and indexed

    - `def __getattr__(self, *args)`
    - `def __call__(self, *args, **kwargs)`
    - `def __getitem__(self, *args)`
    - `def __bool__(self)`

- `def tonull(x)`
    Convert `None` to `null`

- `def get_class(nm, *fld_names, **flds)`
    Dynamically create a class, optionally inheriting from `sup`, containing `fld_names`

- `def mk_class(nm, *fld_names, **flds)`
 

In [ ]:
#|hide
#Test that args and kwargs appear in the markdown doc
assert '**kwargs' in [x for x in _lns if 'def urlopen' in x][0]
_noops_func = [x for x in _lns if 'def noops' in x][0]
assert '**kwargs' in _noops_func and '*args' in _noops_func

## Write markdown to file

We can generate our list of symbols as a markdown file like so:

In [ ]:
#|export
@call_parse
def pysym2md(package_name:Param("Name of the Python package", str),
             include_no_docstring:Param("Include symbols without docstrings?", store_true)=False,
             verbose:Param("Turn on verbose logging?", store_false)=True,
             output_file:Param("The output file", str)='filelist.md'
                         ):
    "Generate a list of symbols corresponding to a python package in a markdown format."
    markdown_content = generate_markdown(package_name, include_no_docstring, verbose)
    Path(output_file).write_text(markdown_content)
    print(f"Documentation generated in {output_file}")

In [ ]:
pysym2md('fastcore')

Processing module: fastcore._modidx
No public symbols found in fastcore._modidx
Processing module: fastcore._nbdev
No public symbols found in fastcore._nbdev
Processing module: fastcore.all
No public symbols found in fastcore.all
Processing module: fastcore.basics
Processing module: fastcore.dispatch
Processing module: fastcore.docments
Processing module: fastcore.docscrape
Processing module: fastcore.foundation
Processing module: fastcore.imghdr
Processing module: fastcore.imports
Processing module: fastcore.meta
Processing module: fastcore.nb_imports
No public symbols found in fastcore.nb_imports
Processing module: fastcore.net
Processing module: fastcore.parallel
Processing module: fastcore.py2pyi
Processing module: fastcore.script
Processing module: fastcore.shutil
No public symbols found in fastcore.shutil
Processing module: fastcore.style
Processing module: fastcore.test
Processing module: fastcore.transform
Processing module: fastcore.utils
No public symbols found in fastcore.ut

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()